In [1]:
# https://github.com/Azure-Samples/openai/blob/main/Basic_Samples/Chat/chatGPT_managing_conversation.ipynb
!pip install --upgrade openai

In [2]:
! pip install tiktoken

In [3]:
# import os module & the OpenAI Python library for calling the OpenAI API
# please make sure you have installed required libraries via pip install -r requirements.txt
import os
import openai
import json

In [4]:
from dotenv import load_dotenv

# **BURAYI .env DOSYANIZIN YERI OLACAK SEKILDE DEGISTIRIN***
dotenv_path = 'C:/Users/cbark/key.env'
is_loaded = load_dotenv(dotenv_path)
print("Is environment variables loaded correctly? ", is_loaded)

Is environment variables loaded correctly?  True


In [5]:
# Load config values
    
# Setting up the deployment name
chatgpt_model_name = os.getenv('CHATGPT_MODEL')

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = os.getenv('OPENAI_API_BASE')

# Currently Chat Completions API have the following versions available: 2023-03-15-preview
openai.api_version = os.getenv('OPENAI_API_VERSION')

In [6]:
# System Message
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant.


In [7]:
# Define helper functions

import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [8]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = openai.ChatCompletion.create(
        engine=chatgpt_model_name,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response['choices'][0]['message']['content']

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

In [9]:
# Start the conversation

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "Renkli çoraplar satan bir şirket için isim öner."

In [10]:
# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [11]:
token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

Token count: 38


In [12]:
max_response_tokens = 500

response = send_message(messages, chatgpt_model_name, max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[SYSTEM]
You are a helpful assistant.

[USER]
Renkli çoraplar satan bir şirket için isim öner.

[ASSISTANT]
Renkli Çorapçı veya Çorap Rengarenk olabilir. Ayrıca, renkli çorapların enerjik ve eğlenceli bir ürün olduğunu düşünürsek, isim olarak "Renkli Hareket" veya "Çorap Şenliği" gibi seçenekler de düşünebilirsiniz.

